# Setup

In [1]:
CASSANDRA_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

CASSANDRA_CLUSTER_NAME = "cassandra-cluster.mavasbel.vpn.itam.mx"
CASSANDRA_TOTAL_NODES = 3

CASSANDRA_NODE_IPS = ["10.15.20.2"] * CASSANDRA_TOTAL_NODES
CASSANDRA_NODE_NAMES = [f"cassandra-node-{i+1}" for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_HOSTNAMES = [
    f"{CASSANDRA_NODE_NAMES[i]}.mavasbel.vpn.itam.mx"
    for i in range(CASSANDRA_TOTAL_NODES)
]
CASSANDRA_NODE_GOSSIP_PORTS = [7000 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_RPC_PORTS = [9040 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_SSL_GOSSIP_PORTS = [7500 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_JMX_PORTS = [7200 + (i + 1) for i in range(0, CASSANDRA_TOTAL_NODES)]

CASSANDRA_CA_CERT_PASSWORD = "cassandra_cluster_ca_cert_passowrd"
CASSANDRA_NODE_CERT_PASSWORD = "cassandra_cluster_cert_passowrd"
CASSANDRA_INIT_USER = "cassandra"
CASSANDRA_INIT_PASSWORD = "cassandra"

CASSANDRA_WORKDIR = "/var/lib/cassandra"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR = os.path.join(LOCALHOST_WORKDIR, "cluster_certs")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("CQLENG_ALLOW_SCHEMA_MANAGEMENT", "True")

'True'

# Session creation

In [3]:
from cassandra.cluster import Cluster
from cassandra.connection import DefaultEndPoint
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

cassandra_nodes = [
    DefaultEndPoint(CASSANDRA_NODE_HOSTNAMES[j], CASSANDRA_NODE_RPC_PORTS[j])
    for j in range(CASSANDRA_TOTAL_NODES)
]
print(
    f"🔗 Connecting to: {[f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes]}"
)

auth_provider = PlainTextAuthProvider(
    username=CASSANDRA_INIT_USER, password=CASSANDRA_INIT_PASSWORD
)
cluster = Cluster(contact_points=cassandra_nodes, auth_provider=auth_provider)

session = cluster.connect()
session.row_factory = dict_factory
print(f"✅ Connected to cluster: {cluster.metadata.cluster_name}")
print(f"🌐 Nodes found: {len(cluster.metadata.all_hosts())}")

🔗 Connecting to: ['cassandra-node-1.mavasbel.vpn.itam.mx:9041', 'cassandra-node-2.mavasbel.vpn.itam.mx:9042', 'cassandra-node-3.mavasbel.vpn.itam.mx:9043']
✅ Connected to cluster: cassandra-cluster.mavasbel.vpn.itam.mx
🌐 Nodes found: 3


In [4]:
import pprint

pprint.pprint(
    [cassandra_host.__dict__ for cassandra_host in cluster.metadata.all_hosts()]
)

[{'_datacenter': 'dc1',
  '_rack': 'rack1',
  'broadcast_address': '10.15.20.2',
  'broadcast_port': 7002,
  'broadcast_rpc_address': '10.15.20.2',
  'broadcast_rpc_port': 9042,
  'conviction_policy': <cassandra.policies.SimpleConvictionPolicy object at 0x000001AAE1FD6E90>,
  'dse_version': None,
  'dse_workload': None,
  'dse_workloads': None,
  'endpoint': <DefaultEndPoint: cassandra-node-2.mavasbel.vpn.itam.mx:9042>,
  'host_id': UUID('4f1a33f3-5359-4367-9073-3507a5e80be8'),
  'is_up': True,
  'listen_address': '172.19.0.3',
  'listen_port': 7002,
  'lock': <unlocked _thread.RLock object owner=0 count=0 at 0x000001AAC39AB600>,
  'release_version': '5.0.6'},
 {'_datacenter': 'dc1',
  '_rack': 'rack1',
  'broadcast_address': '10.15.20.2',
  'broadcast_port': 7001,
  'broadcast_rpc_address': '10.15.20.2',
  'broadcast_rpc_port': 9041,
  'conviction_policy': <cassandra.policies.SimpleConvictionPolicy object at 0x000001AAE51902B0>,
  'dse_version': None,
  'dse_workload': None,
  'dse_wo

In [5]:
# from cassandra.cluster import ResultSet

# peers_v2: ResultSet = session.execute("SELECT * FROM system.peers_v2")
# locals: ResultSet = session.execute("SELECT * FROM system.local")

# for peer_v2 in peers_v2:
#     print(" | ".join([f"{col}: {peer_v2[col]}" for col in peers_v2.column_names]))
# print("=" * 100)
# for local in locals:
#     print(" | ".join([f"{col}: {local[col]}" for col in locals.column_names]))
# print("=" * 100)

### Keyspace

In [6]:
keyspace_name = "generic_analytics"
if CASSANDRA_START_FROM_SCRATCH:
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace_name}")

In [7]:
keyspace_name = "generic_analytics"
session.execute(
    f"""
CREATE KEYSPACE IF NOT EXISTS {keyspace_name} 
WITH replication = {{
    'class': 'NetworkTopologyStrategy', 
    'dc1': {CASSANDRA_TOTAL_NODES}
}}
"""
)
session.set_keyspace(keyspace_name)

# Create Table
# Partition Key: city (distributes data)
# Clustering Column: user_id (sorts data within city)
session.execute(
    """
CREATE TABLE IF NOT EXISTS user_metrics (
    city text,
    user_id uuid,
    username text,
    session_duration int,
    last_access timestamp,
    PRIMARY KEY (city, user_id)
);
"""
)

### Insert

In [8]:
from faker import Faker

fake = Faker()

In [9]:
import uuid
from datetime import datetime
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, BatchType

# 1. Prepare your statement outside the loop
query = """
INSERT INTO user_metrics (city, user_id, username, session_duration, last_access)
VALUES (?, ?, ?, ?, ?)
"""
prepared = session.prepare(query)
# prepared.consistency_level = ConsistencyLevel.QUORUM
# statement = SimpleStatement(query, consistency_level=ConsistencyLevel.QUORUM)

# 2. Create the Batch object
# LOGGED ensures atomicity but adds disk overhead, UNLOGGED is faster
batch_records = 200
batch = BatchStatement(batch_type=BatchType.LOGGED)
batch.consistency_level = ConsistencyLevel.QUORUM
print(f"Preparing batch of {batch_records} records...")
for _ in range(batch_records):
    batch.add(
        prepared,
        (
            fake.city(),
            uuid.uuid4(),
            fake.user_name(),
            fake.random_int(min=1, max=3600),
            datetime.now(),
        ),
    )

# 3. Execute the entire batch at once
session.execute(batch)
print("Batch successfully committed to the cluster.")

Preparing batch of 200 records...
Batch successfully committed to the cluster.


### Query

In [10]:
from cassandra.cluster import ResultSet
from typing import cast

print(cast(ResultSet, session.execute("SELECT count(*) FROM user_metrics")).one())

{'count': 200}


In [11]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT * FROM user_metrics LIMIT 100")
for row in rows.current_rows:
    print(row)

{'city': 'Lake Matthew', 'user_id': UUID('4e054c5e-0ffd-496b-b85f-0e917ae041c1'), 'last_access': datetime.datetime(2026, 1, 10, 21, 44, 8, 263000), 'session_duration': 159, 'username': 'wallacematthew'}
{'city': 'New Lindachester', 'user_id': UUID('8656f89a-eec2-499b-8e9c-b3349e3ccd15'), 'last_access': datetime.datetime(2026, 1, 10, 21, 44, 8, 253000), 'session_duration': 1144, 'username': 'david19'}
{'city': 'Lindaville', 'user_id': UUID('2acd7d67-ff61-43ae-90e5-d56ead065b86'), 'last_access': datetime.datetime(2026, 1, 10, 21, 44, 8, 272000), 'session_duration': 3067, 'username': 'keiththomas'}
{'city': 'Novakberg', 'user_id': UUID('0fbcb08d-884f-40f1-9c4b-11c022e3830a'), 'last_access': datetime.datetime(2026, 1, 10, 21, 44, 8, 253000), 'session_duration': 1325, 'username': 'davistamara'}
{'city': 'Lake Raymondland', 'user_id': UUID('a63ba14b-dcbe-4b57-b043-9b66e3f1af19'), 'last_access': datetime.datetime(2026, 1, 10, 21, 44, 8, 252000), 'session_duration': 1329, 'username': 'ronald90

In [12]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT DISTINCT city FROM user_metrics", [])
for row in rows.current_rows:
    print(row)

{'city': 'Lake Matthew'}
{'city': 'New Lindachester'}
{'city': 'Lindaville'}
{'city': 'Novakberg'}
{'city': 'Lake Raymondland'}
{'city': 'Port Jessica'}
{'city': 'Boydton'}
{'city': 'East Anthony'}
{'city': 'Ericville'}
{'city': 'Port Susanmouth'}
{'city': 'Davidsonmouth'}
{'city': 'Shawnton'}
{'city': 'East Amandaside'}
{'city': 'East Angelabury'}
{'city': 'Grayfort'}
{'city': 'Elizabethstad'}
{'city': 'New Hailey'}
{'city': 'Shellyborough'}
{'city': 'Jessemouth'}
{'city': 'Port Tamara'}
{'city': 'Kellybury'}
{'city': 'Owensshire'}
{'city': 'South Steven'}
{'city': 'North Amberhaven'}
{'city': 'Rileymouth'}
{'city': 'New Jerry'}
{'city': 'West Johnfort'}
{'city': 'Garrettville'}
{'city': 'Garciaborough'}
{'city': 'South Hollyfort'}
{'city': 'Lake Breanna'}
{'city': 'Mooreland'}
{'city': 'Jameschester'}
{'city': 'Baxterfurt'}
{'city': 'East Matthewfurt'}
{'city': 'Lake Kevinstad'}
{'city': 'Lake Danielleton'}
{'city': 'Carlsonview'}
{'city': 'Kennethland'}
{'city': 'Drewton'}
{'city': 

### Find nodes storing random data

In [13]:
import random
import pprint

random_token = random.randint(-9223372036854775808, 9223372036854775807)
query = "SELECT * FROM user_metrics WHERE token(city) >= %s LIMIT 1"
row: dict = cast(ResultSet, session.execute(query, [random_token])).one()
print(f"Random user_metric: {pprint.pformat(row)}")

prepared = session.prepare("SELECT * FROM user_metrics WHERE city=? AND user_id=?")
bound = prepared.bind([row['city'], row['user_id']])
routing_key = bound.routing_key
nodes = cluster.metadata.get_replicas(keyspace_name, routing_key)

print(f"Nodes storing '{row['city']}':")
for node in nodes:
    print(f" - Host: {node.address}, Gossip Port: {node.broadcast_port}")

Random user_metric: {'city': 'East Susan',
 'last_access': datetime.datetime(2026, 1, 10, 21, 44, 8, 250000),
 'session_duration': 1734,
 'user_id': UUID('de1129d0-73f4-46fe-9dcc-d1d89a423623'),
 'username': 'amullen'}
Nodes storing 'East Susan':
 - Host: 10.15.20.2, Gossip Port: 7001
 - Host: 10.15.20.2, Gossip Port: 7003
 - Host: cassandra-node-2.mavasbel.vpn.itam.mx, Gossip Port: 7002


### ORM-like

In [14]:
from typing import cast
from cassandra.cqlengine import columns
from cassandra.cqlengine.query import ModelQuerySet
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table, create_keyspace_network_topology
from cassandra.cqlengine import connection

# 1. Connect the engine
connection.set_session(session)


# 2. Define your "Generic" Model
class UserMetrics(Model):
    __table_name__ = "user_metrics"

    # FIRST primary_key=True is the Partition Key
    # city = columns.Text(primary_key=True)
    city = columns.Text(primary_key=True, partition_key=True)

    # SECOND primary_key=True is the Clustering Key
    user_id = columns.UUID(primary_key=True, default=uuid.uuid4)

    # Attributes (Data)
    username = columns.Text(index=True)
    session_duration = columns.Integer()
    last_access = columns.DateTime()


# 3. Create Keyspace and Sync Table (Equivalent to CREATE TABLE)
create_keyspace_network_topology(keyspace_name, {"dc1": CASSANDRA_TOTAL_NODES})
sync_table(UserMetrics)

In [15]:
# 4. Use it like an ORM
# Create UserMetrics
new_metric: UserMetrics = UserMetrics.create(
    city=fake.city(),
    username=fake.name(),
    session_duration=120,
    last_access=datetime.now(),
)
new_metric.save()
print(f"Saved user_metris: {new_metric}")


# 5. Query UserMetrics
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    print(f"Query user_metris: {user_metric}")

Saved user_metris: UserMetrics <city=Russellburgh, user_id=6d31dbd2-797b-4864-9ae4-c8a646e3e365>
Query user_metris: UserMetrics <city=Lake Matthew, user_id=4e054c5e-0ffd-496b-b85f-0e917ae041c1>
Query user_metris: UserMetrics <city=New Lindachester, user_id=8656f89a-eec2-499b-8e9c-b3349e3ccd15>
Query user_metris: UserMetrics <city=Lindaville, user_id=2acd7d67-ff61-43ae-90e5-d56ead065b86>
Query user_metris: UserMetrics <city=Novakberg, user_id=0fbcb08d-884f-40f1-9c4b-11c022e3830a>
Query user_metris: UserMetrics <city=Lake Raymondland, user_id=a63ba14b-dcbe-4b57-b043-9b66e3f1af19>
Query user_metris: UserMetrics <city=Port Jessica, user_id=139a007b-f0bf-4e7a-b449-9457b5266692>
Query user_metris: UserMetrics <city=Boydton, user_id=c3ab1226-2d57-43c1-9926-947152ee7042>
Query user_metris: UserMetrics <city=East Anthony, user_id=a270859c-3dba-4f39-a462-ec6f12fe82ec>
Query user_metris: UserMetrics <city=Ericville, user_id=9f304fdb-707f-4dc6-8961-6425ec1895f6>
Query user_metris: UserMetrics <cit

### Insert/Create

In [16]:
from typing import cast

cast(ModelQuerySet, UserMetrics.ttl(86400)).create(
    city=fake.city(),
    username=fake.user_name(),
    session_duration=fake.random_int(10, 1000),
    last_access=fake.date_time(),
)

UserMetrics(city='West Melissa', user_id=UUID('8931fe35-95ee-40d3-b5ac-11d8172df24a'), username='wyattknox', session_duration=117, last_access=datetime.datetime(2000, 11, 5, 11, 52, 3))

In [17]:
from typing import cast
from cassandra.cqlengine.query import BatchQuery
from cassandra.cqlengine.query import ModelQuerySet

with BatchQuery() as b:
    for _ in range(100):
        cast(ModelQuerySet, UserMetrics.batch(b)).create(
            city=fake.city(),
            username=fake.user_name(),
            session_duration=fake.random_int(10, 1000),
            last_access=fake.date_time(),
        )

### Delete

In [18]:
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    user_metric.delete()
    print(f"Delete user_metris: {user_metric}")

Delete user_metris: UserMetrics <city=Lake Matthew, user_id=4e054c5e-0ffd-496b-b85f-0e917ae041c1>
Delete user_metris: UserMetrics <city=Phillipsmouth, user_id=10a96364-ece0-40e6-9d22-c10d4a26d5cc>
Delete user_metris: UserMetrics <city=Christopherfurt, user_id=4385f024-3feb-4d7b-95ff-d5004b606534>
Delete user_metris: UserMetrics <city=New Lindachester, user_id=8656f89a-eec2-499b-8e9c-b3349e3ccd15>
Delete user_metris: UserMetrics <city=Lindaville, user_id=2acd7d67-ff61-43ae-90e5-d56ead065b86>
Delete user_metris: UserMetrics <city=North Daniel, user_id=163d60b5-54cb-4a2b-bb36-8a423fede25e>
Delete user_metris: UserMetrics <city=Novakberg, user_id=0fbcb08d-884f-40f1-9c4b-11c022e3830a>
Delete user_metris: UserMetrics <city=Kevinland, user_id=34d97a6d-8dec-4af0-8137-84c72a799faa>
Delete user_metris: UserMetrics <city=North Christina, user_id=953c7953-358c-45c1-bf38-d53318aa8c98>
Delete user_metris: UserMetrics <city=New Robertmouth, user_id=9d26650f-f251-4878-9866-79fedfab6faa>
Delete user_me

In [19]:
# from cassandra.cqlengine import connection

# # This isn't a compaction, but it forces RAM data to disk so the background compaction process can see it.
# connection.execute(f"ALTER TABLE {keyspace_name}.user_metrics WITH gc_grace_seconds = 10")